#### 데이터 분석 연습 
- 데이터 확인  
    - 데이터 구조 확인 
        - 필요에 따라 컬럼 추가 ( ex) Age_Category )
    - 결측 데이터 처리 
        - < 다양한 방법 존재 >
        - #1) 가장 많은 값으로 채움
        - #2) 연관있는 데이터 찾아, 해당 데이터 행들이 갖는 값의 평균을 결측치에 입력
             - ex) Age : Sex Pclass가 연관성이 있음 -> 공통값을 갖는 데이터의 Age 이용
        - #3) 결측값이 많은 컬럼 -> 삭제 
        - #4) 동일한 값으로 결측 데이터 채우기
- 인코딩 
    - Label Encoding 
    - One Hot Encoding 
- 데이터 스케일링 
    - 표준화 -> 아웃라이어 제거 위함  
        - 평균으로 부터 떨어져 있는 정도를 판단
    - 정규화 -> 데이터의 상대적 크기에 대한 영향을 줄임 
        - 0 ~ 1 사이의 값을 갖도록 정규화 
- 모델 적용 
    - 학습데이터 / 테스트데이터 분리
    - 모델 적용 
- 교차 검증

#### 문제 목표 
1) 타이타닉 탑승자의 생존여부 분류 

#### 데이터 확인 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/titanic.csv') # 데이터 읽어오고

In [3]:
df.head(3) # 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [4]:
df.shape # 컬럼, 행 확인

(891, 12)

In [5]:
df.info() 
# 결측값이 있는 값 확인 
# Name, Sex, Ticket : 문자열 -> 처리 방법 생각 
# 1) 티켓/이름/성별이 생존과 관련이 있었나?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df.describe() # NaN 값, 카테고리 데이터 제외된 값 

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### Survived와 다른 컬럼 확인 

In [7]:
df[df['Survived']==1].Sex.value_counts()
# 여자가 많이 살았다

female    233
male      109
Name: Sex, dtype: int64

In [8]:
# 나이 별 카테고리화 
def get_age_cat(age):
    cat =''
    if age <= 5:
        cat = 'Baby'
    elif age <= 12:
        cat = 'Child'
    elif age <= 18:
        cat = 'Teenager'
    elif age <= 25:
        cat = 'Student'
    elif age <= 35:
        cat = 'Young Adult'
    elif age <= 65:
        cat = 'Adult'
    else:
        cat = 'Old Adult'
    
    return cat
        
df['Age_cat'] = df['Age'].apply(lambda x: get_age_cat(x))
df[['Age','Age_cat']].head(3)

,Age,Age_cat
0,22.0,Student
1,38.0,Adult
2,26.0,Young Adult


In [9]:
df[df['Survived']==0].Age_cat.value_counts().sort_values()

Baby            13
Child           16
Teenager        40
Student        108
Young Adult    113
Adult          127
Old Adult      132
Name: Age_cat, dtype: int64

###### groupby([ ])['Column'].value_counts()

In [10]:
df.groupby('Sex').Survived.value_counts().sort_values()

Sex     Survived
female  0            81
male    1           109
female  1           233
male    0           468
Name: Survived, dtype: int64

In [11]:
df[(df['Survived'] == 1) & (df['Sex'] == 'female')]['Survived'].count()
# 여자이면서 살아남은 사람 : 233

233

In [12]:
df[(df['Survived'] == 1) & (df['Sex'] == 'male')]['Survived'].count()
# 남자이면서 살아남은 사람 : 109

109

In [13]:
df[df['Survived']==0]['Survived'].count()

549

In [14]:
df[df['Survived']==1]['Survived'].count()
# 342명 살아남았음 

342

In [15]:
df[(df['Survived'] == 1) & (df['Sex'] == 'male')]['Age_cat'].value_counts()


Young Adult    30
Adult          25
Old Adult      17
Baby           15
Student        13
Child           6
Teenager        3
Name: Age_cat, dtype: int64

In [16]:
df[(df['Survived'] == 1) & (df['Sex'] == 'female')]['Age_cat'].value_counts().sort_values()

Child           3
Baby           16
Teenager       27
Old Adult      36
Student        41
Young Adult    53
Adult          57
Name: Age_cat, dtype: int64

In [35]:
df.groupby(['Age_cat', 'Pclass']).size()

Age_cat      Pclass
Adult        1         100
             2          48
             3          61
Baby         1           3
             2          13
             3          28
Child        1           1
             2           4
             3          20
Old Adult    1          34
             2          13
             3         138
Student      1          26
             2          36
             3         100
Teenager     1          12
             2          12
             3          46
Young Adult  1          40
             2          58
             3          98
dtype: int64

In [46]:
pd.DataFrame(df.groupby(['Sex', 'Age_cat']).sum()['Survived'])

Survived
Sex    Age_cat              
female Adult              57
       Baby               16
       Child               3
       Old Adult          36
       Student            41
       Teenager           27
       Young Adult        53
male   Adult              25
       Baby               15
       Child               6
       Old Adult          17
       Student            13
       Teenager            3
       Young Adult        30

In [45]:
df[(df['Sex']=='female') & (df['Age_cat'] == 'Adult')]['Survived'].sum()

57

In [18]:
df['Age_cat'].value_counts()

Adult          209
Young Adult    196
Old Adult      185
Student        162
Teenager        70
Baby            44
Child           25
Name: Age_cat, dtype: int64

#### 결측치 확인

In [19]:
df.isnull().sum()
# 결측치 갯수 확인
# 나이, 객실, 출항지 결손 확인 

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Age_cat          0
dtype: int64

#### 결측치 처리 
- 삭제 / 평균값 채워 넣는 방법 - ①
- 사람의 특징에 따라 채워넣는 방법 - ②

In [20]:
df.drop('Embarked', axis=1, inplace = True)

In [21]:
df.info()
# Age, Cabin 처리 필요 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Age_cat      891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [22]:
df.Age.fillna(df['Age'].mean(), inplace = True)

In [23]:
df.Age.isnull().sum()

0

In [24]:
df.Cabin.fillna('S', inplace = True)

In [25]:
df.Cabin.isnull().sum()

0

In [26]:
df.info()
# Name, Sex, Tickt, Cabin 라벨링 필요

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Age_cat      891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [27]:
df.Ticket.value_counts()

1601                 7
347082               7
CA. 2343             7
3101295              6
347088               6
                    ..
STON/O2. 3101290     1
347087               1
349245               1
693                  1
SOTON/O.Q. 392078    1
Name: Ticket, Length: 681, dtype: int64

###### 티켓 드롭? 남겨?

In [28]:
print('티켓 유일한 값 : ', len(df.Ticket.unique()),'전체 티켓 : ', df.Ticket.count() )
df[df.duplicated(subset='Ticket') & (df['Pclass'] != 1)][['Ticket', 'Name','Survived','Pclass']].sort_values(by='Ticket').head(30)
# 같은 티켓이면 가족인가? 그리고 같은 티켓을 가진사람들끼리는 죽었나? 
# 티켓이 같더라도 사는 것과는 관련이 없는 것 같다. 
# 티켓 삭제 

티켓 유일한 값 :  681 전체 티켓 :  891


,Ticket,Name,Survived,Pclass
117,11668,"Turpin, Mr. William John Robert",0,2
826,1601,"Lam, Mr. Len",0,3
692,1601,"Lam, Mr. Ali",1,3
169,1601,"Ling, Mr. Lee",0,3
643,1601,"Foo, Mr. Choong",1,3
509,1601,"Lang, Mr. Fang",1,3
838,1601,"Chip, Mr. Chang",1,3
754,220845,"Herman, Mrs. Samuel (Jane Laver)",1,2
340,230080,"Navratil, Master. Edmond Roger",1,2
193,230080,"Navratil, Master. Michel M",1,2


In [29]:
df.drop('Ticket', axis = 1, inplace = True)

###### Fare 드롭? 남겨?

In [30]:
df[df.duplicated(subset='Fare') & df.Fare.notnull() & df.Fare != 0][['Fare', 'Name', 'Survived']].sort_values(by='Fare').head(20)
# 같은 돈을 주고 탔으면 가족일까?
# 티켓과 마찬가지인 듯 하다 
# 지우자
df.drop('Fare', axis = 1, inplace = True)

###### 결측치 Threshold 주기 

df = df.dropna(axis=1, thresh=100)


###### 결측치 깔끔하게 삭제하기
- df.dropna(subset='Column', how = 'any' )

#### object형 데이터 라벨링 